#### Author: Michael Polinsky

# Project 2: Epicurious Recipe Data, PCA, and Logistic Regression

#### Imports

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sbn
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
from sklearn.decomposition import PCA
from sklearn.metrics import matthews_corrcoef as matts_c
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_score, recall_score

# I usually like to have a cell that allows you to pull the data from a URL but I was unable to get such a cell working on this project.  I emailed you about it and you just said to download the data and not worry about it.  As long as the recipe data is in the same folder as the notebook it will run, but you may have to run it locally.

## Load Data

In [3]:
all_data = pd.read_csv('epi_r.csv')

FileNotFoundError: ignored

## Describe data

In [ ]:
all_data.describe()

## Look at rating

In [ ]:
all_data['rating'].hist()

### There are far more scores at the top of the range than the bottom.

##  Plot 22-minute meals and #cakeweek

In [ ]:
all_data['#cakeweek'].hist()

### These appear not to have any 1 values.

In [ ]:
all_data['spring'].hist()

In [ ]:
len(all_data[all_data['#cakeweek'] == 1])

In [ ]:
len(all_data[all_data['22-minute meals'] == 1])

## I did more visualizations than I want to fill the notebook with here but I discovered a set of features that were not so sparse, and decided to add them.

In [ ]:
len(all_data[all_data['winter'] == 1])

In [ ]:
len(all_data[all_data['spring'] == 1])

In [ ]:
len(all_data[all_data['summer'] == 1])

In [ ]:
len(all_data[all_data['fall'] == 1])

In [ ]:
len(all_data[all_data['bake'] == 1])

In [ ]:
len(all_data[all_data['peanut free'] == 1])

## Making a smaller dataset.  Looking at histograms and a covariance matrix graph led me to try the additional categories.

In [ ]:
bakers_dozen_columns = ['calories','fat','sodium','protein', 'winter', 'spring', 'summer', 'fall', 'peanut free', 'bake', '#cakeweek', '22-minute meals', 'rating'] # 13 columns

In [ ]:
small_data = all_data.loc[:,bakers_dozen_columns]

# Drop duplicates and outliers from small data

In [ ]:
small_data_no_dups = small_data.drop_duplicates()

In [ ]:
small_data_no_dups

# Dropping outliers 

In [ ]:
drop_list = small_data_no_dups[(small_data_no_dups['fat'] > 300) | (small_data_no_dups['calories'] > 5000) | (small_data_no_dups['sodium'] > 5000) | (small_data_no_dups['protein'] > 300) ].index 

In [ ]:
small_data_dropped = small_data_no_dups.drop(drop_list)

# Drop NA

In [ ]:
df = small_data_dropped.dropna()

## The number of instances with 1s for '#cakeweek' and '22-minute meals' is dwindling.

In [ ]:
len(df[df['#cakeweek'] == 1])

In [ ]:
len(df[df['22-minute meals'] == 1])

# Train/Test Splits

In [ ]:
xList = ['fat','calories','protein','sodium','bake','peanut free'] 
yList = ['rating','#cakeweek','22-minute meals', 'winter','spring','summer','fall']

In [ ]:
trainX, testX, trainY, testY = train_test_split(df[xList], df[yList], train_size = 0.8, 
                                                          test_size = 0.2, 
                                                          random_state = 100)

### And now we have the following 1-0 splits in our Y sets. (I realize that in practice we never look at the test data.)

In [ ]:
len(trainY[trainY['22-minute meals'] == 1])

In [ ]:
len(trainY[trainY['22-minute meals'] == 0])

In [ ]:
len(testY[testY['22-minute meals'] == 1])

### This means #cakeweek will have all 0s in the test data, and the regressor will only see 4 1s vs 11,329 0s.
### The same numbers for '22-minute meals' are 10 1s vs 11323 0s  in the training set and only 5 1s in the test.

# Scale the data

### I'm using a MaxAbsScaler because I read that it maintains sparseness in data.  I wondered if that would be a good thing or not, so I tried using StandardScaler instead and found that the classification was worse, and the PCA required twice as many components to account for 95% of the variance compared to MaxAbsScaler.  

#### Scale training X

In [ ]:
# also try MinMaxScaler
scaler = MaxAbsScaler().fit(trainX)
scaled_data = scaler.transform(trainX)
sdf1 = pd.DataFrame(scaled_data)
sdf1

#### Scale test X

In [ ]:
scaler_test = MaxAbsScaler().fit(testX)
scaled_data_test = scaler_test.transform(testX)
sdf2 = pd.DataFrame(scaled_data_test)

## show distributions before and after scaling

In [ ]:
#Raw Data
df.hist(bins=50, figsize=(20,15))

#Scaled Data:
sdf1.hist(bins=50, figsize=(20,15))


## Pearson coefficient

### Using a loop to accumulate the r's for the features in X

In [ ]:
pearson_rs = []
for col, item in sdf1.iteritems():
    pearson_rs.append(sp.stats.pearsonr(item, trainY['rating']))

### The highest r is fat at 0.125, then calories at 0.112, then protein at .116, peanut free at 0.081, sodium at 0.079, and bake at 0.074.

In [ ]:
for each in pearson_rs:
    print(each[0])

## Apply PCA

### Apply with fit_transform and specify 95% variance.  

In [ ]:
pca = PCA()
x_pca= pca.fit_transform(sdf1)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1

### See that the resulting dataseet has the same shape.

In [ ]:
print ('Scaled data shape: ', sdf1.shape)
print ('Transformed data shape: ', x_pca.shape)
print ('Explained variance: ', pca.explained_variance_ratio_)

### The first PC accounts for 53% of the variance, the 2nd 37%, the 3rd 6%, and the rest 2% and less than 0.00%.

# Elbow Plot

### Shows that we reach 95% varriance with 2 components.
### When I used StandardScaler instead of MaxAbsScaler, 4 components were required.

In [ ]:
# The elbow plot of the explained variance

plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('No. of Components')
plt.ylabel('Explained Variance')
plt.title('Elbow Plot')
plt.show()

# Logistic Regression

### Here I've settled on predicting 'winter' using calories.  Strangely, I got slightly better classification by training on 'calories' and predicting using 'bake', but that seems like nonsense so I left it alone.

In [ ]:
x_col = sdf1[1].values.reshape(-1,1)
X = x_col
Y = trainY['winter']

# Perform logistic regression for the binary output
classifier = LogisticRegression()
classifier.fit(X, Y)

# Confusion matrix

In [ ]:
Y_pred = classifier.predict(sdf2[1].values.reshape(-1,1)) 
labels = [0, 1]
cm = confusion_matrix(testY['winter'], Y_pred, labels=labels)

print ('#Total cases', df.shape)
print ('#Training and testing cases', sdf1.shape, sdf2.shape)
print ('\nConfusion matrix:')
pd.DataFrame(cm, index=labels, columns=labels)

In [ ]:
print('Precision:',precision_score(testY['winter'],Y_pred,pos_label=1))
print('Recall:',recall_score(testY['winter'],Y_pred,pos_label=1))

### That's not very good, but it's better than I was able to get using StandardScaler, which didn't predict any classes at all. Still, the coin-flip predictor is better than this, assuming a fair coin.

# Feature engineering 'healthy'

## Create new feature healthy: protein + calories / fat + sodium.  My original formula had protein in the numerator and the other nutrition features in the denominator. To try and get better results I changed it to this which turns out to be no better, but the thought was that healthy foods can contain healthy fats, so just dividing out fat content wouldn't necessarily correspond to healthy foods.

In [ ]:
healthy = pd.Series(df.loc[:,'protein'] + df.loc[:,'fat'] / (df.loc[:,'calories'] + df.loc[:,'sodium']))

### The code below generates a warning but seemingly should not.  I just run it again to get rid of the warning.
### The error says: 'Try using .loc[row_indexer,col_indexer] = value instead', which I do, unless I'm just blind to a mistake.

In [ ]:
df.loc[:,'healthy'] = healthy # Shouldn't be getting this warning.  

In [ ]:
healthy

## We need to do a little bit of processing on the data set because of the new feature

In [ ]:
df[df['healthy'].isnull()]

#### Need to get rid of any 'healthy' NaNs.  After running this rerunning the last cell confirms the drop.

In [ ]:
df = df.dropna()

### There are 20 instances with value inf for healthy, which need to be dropped.

In [ ]:
infinite_health = df.iloc[df.values==np.inf]

In [ ]:
df = df.drop(infinite_health.index)

## The histogram for healthy.

In [ ]:
df['healthy'].hist()

## Creating new splits from the processed data in df and training with the new feature.

In [ ]:
xList_h = ['fat','calories','protein','sodium','bake','peanut free', 'healthy'] 
yList_h = ['rating','#cakeweek','22-minute meals', 'winter','spring','summer','fall']

In [ ]:
trainX_h, testX_h, trainY_h, testY_h = train_test_split(df[xList_h], df[yList_h], train_size = 0.8, 
                                                          test_size = 0.2, 
                                                          random_state = 100)

# Scale

#### Scale training X

In [ ]:
scaler = MaxAbsScaler().fit(trainX_h)
scaled_data = scaler.transform(trainX_h)
sdf1_h = pd.DataFrame(scaled_data)
sdf1_h

#### Scale test X

In [ ]:
scaler_test = MaxAbsScaler().fit(testX_h)
scaled_data_test = scaler_test.transform(testX_h)
sdf2_h = pd.DataFrame(scaled_data_test)

## Pearson coefficient for 'healthy', 'rating'

In [ ]:
healthy_r = sp.stats.pearsonr(trainX_h['healthy'], trainY_h['rating'])

In [ ]:
healthy_r[0]

### Not highly correlated, but not surprising, becuase the goal was to create a feature 'healthy' and then see how correlated it was, rather than trying to create any   feature that might be more correlated with rating. I wonder if a feature called 'unhealthy' might have been more likely to lead to a high rating?  

# Prediction with healthy

In [ ]:
pnf_h = sdf1_h[6].values.reshape(-1,1)
X_h = pnf_h
Y_h = trainY_h['winter']

# Split the dataset (569 instances) into the training set (70%) and testing (30%)
# random_state Controls the shuffling applied to the data before applying the split. 
# Pass an int for reproducible output across multiple function calls.
#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.30, random_state = 0)

# Perform logistic regression for the binary output
healthy_classifier = LogisticRegression()
healthy_classifier.fit(X_h, Y_h)

# Confusion matrix

In [ ]:
Y_pred_h = healthy_classifier.predict(sdf2_h[6].values.reshape(-1,1)) 
labels = [0, 1]
cm_h = confusion_matrix(testY_h['winter'], Y_pred_h, labels=labels)

print ('#Total cases', df.shape)
print ('#Training and testing cases', sdf1_h.shape, sdf2_h.shape)
print ('Confusion matrix:')
pd.DataFrame(cm_h, index=labels, columns=labels)

In [ ]:
print('Precision:',precision_score(testY_h['winter'],Y_pred_h, pos_label=1))
print('Recall:',recall_score(testY_h['winter'],Y_pred_h, pos_label=1))

### Not a very good classifier.

# Analysis/Reflections 

### I was not able to create an LR model that was a good predictor on this data. The sparseness of the data proved to be a real problem.  For instance, when I was attempting to predict '#cakeweek' and '22-minute meals' there were few to no instances in the test and training data that had a value of 1, so the model underfit.

### One very interesting part of the lab was discovering the difference that the MaxAbsScaler, which uses the largest absolute value of a feature's values to scale that feature.  Doing so caused a non-classifying model to classify a little, but also led to greater dimensionality reduction.   
 
### I wondered if the problem with 'healthy' was that it was created from all four of the nutrition features, even though not all of them are highly correlated with 'rating', but as in the California Hoousing Data example, it seems like it is generally a possiblilty to create a more highly correlated feature from multiple features that are not highly correlated with the target.

### One thing I decided notto include that I mentioned earlier was that I got not good but better classification results when I fit the LR model to the feature 'calories', predicting 'winter', and then predicted on the test set's 'bake' feature.  I imagine that the seemingly-improved performance was just a ghost becuase I would imagine that predicting on a different feature than you trained a regressor on would be meaningless with unpredictable results.  